In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import time
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')

In [3]:
from june import World 
from june.demography.geography import Geography
from june.demography import Demography
from june.interaction import Interaction
from june.infection import Infection, HealthIndexGenerator
from june.infection.transmission import TransmissionConstant
from june.groups import Hospitals, Schools, Companies, Households, CareHomes, Cemeteries, Universities
from june.groups.leisure import generate_leisure_for_config, Cinemas, Pubs, Groceries
from june.simulator import Simulator
from june.infection_seed import InfectionSeed
from june.policy import Policy, Policies
from june import paths
from june.hdf5_savers import load_geography_from_hdf5
from june.logger.read_logger import ReadLogger
from june.infection.infection import InfectionSelector
from june.world import generate_world_from_hdf5, generate_world_from_geography

No --data argument given - defaulting to:
/home/florpi/JUNE/data
No --configs argument given - defaulting to:
/home/florpi/JUNE/configs


# Initialize world

To initialize a certain world, we need to add the different components we want to have in it. First we specify what super areas (msoa) we want to create. We have included these ones, because they are known to contain hospitals, schools, care homes, and companies.

After creating the geography, we create the different components the worlds need to have such as care homes, companies ...

In [4]:
CONFIG_PATH = "../configs/config_example.yaml"

In [5]:
%%time 

geography = Geography.from_file(
        {
            "super_area": [
                "E02003282",
                "E02001720",
                "E02002560",
                "E02002559",
                "E02004314",
            ]
        }
)

geography.hospitals = Hospitals.for_geography(geography)
geography.schools = Schools.for_geography(geography)
geography.companies = Companies.for_geography(geography)
geography.care_homes = CareHomes.for_geography(geography)
geography.universities = Universities.for_super_areas(geography.super_areas)
world = generate_world_from_geography(geography, include_households=True, include_commute=True)

2020-08-04 15:49:31,856 - june.demography.geography - INFO - There are 129 areas and 5 super_areas in the world.
2020-08-04 15:49:31,863 - june.groups.hospital - INFO - There are 1 hospitals in this geography.
2020-08-04 15:49:31,947 - june.groups.school - INFO - There are 19 schools in this geography.
2020-08-04 15:49:31,974 - june.groups.school - INFO - No school for the age 0 in this world.
2020-08-04 15:49:31,977 - june.groups.school - INFO - No school for the age 1 in this world.
2020-08-04 15:49:31,981 - june.groups.school - INFO - No school for the age 2 in this world.
2020-08-04 15:49:32,299 - june.groups.carehome - INFO - There are 9 care_homes in this geography.
2020-08-04 15:49:44,689 - numexpr.utils - INFO - NumExpr defaulting to 4 threads.
2020-08-04 15:49:49,551 - june.distributors.worker_distributor - INFO - There are 0 who had to be told to stay real
CPU times: user 42.5 s, sys: 1.85 s, total: 44.3 s
Wall time: 45.7 s


## Commute, travel and leisure

In [6]:
world.pubs = Pubs.for_geography(geography)
world.cinemas = Cinemas.for_geography(geography)
world.groceries = Groceries.for_geography(geography)
leisure = generate_leisure_for_config(world, config_filename=CONFIG_PATH)
leisure.distribute_social_venues_to_households(world.households) # this assigns possible social venues to people.

2020-08-04 15:50:20,617 - june.groups.leisure.leisure - INFO - Distributing social venues to households
2020-08-04 15:50:20,659 - june.groups.leisure.leisure - INFO - Distributed in 0 of 16223 households.


We are also going to need some cemeteries...geography.cemeteries = Cemeteries()


In [7]:
world.cemeteries = Cemeteries()

In [8]:
len(world.people)

46784

### If it took a long time to run the previous commands, it might be a good idea to save the world to reuse it later.

In [9]:
world.to_hdf5("world.hdf5")

If we would like to load the world we saved, we just do

In [10]:
world = generate_world_from_hdf5("world.hdf5")

loading world data ...
loading population from hdf5 .
loading companies from hdf5 .
loading households from hdf5 .
restoring world...
restoring people...
done
restoring areas...
done
restoring households...
done
restoring care homes...
done
restoring commute...
done


In [11]:
# and regenerate leisure in case we load it externally
leisure = generate_leisure_for_config(world, CONFIG_PATH)

you have now a beautiful pre-pandemic world. 

# Adding the infection

The module in charge of infecting people is called the ``InfectionSelector``, which gives people a transmission time profile and a symptoms trajectory based on their age and sex (through the health index generator)

In [12]:
health_index_generator = HealthIndexGenerator.from_file(asymptomatic_ratio=0.2)
selector = InfectionSelector.from_file(
        health_index_generator=health_index_generator,
        transmission_config_path=paths.configs_path / 'defaults/transmission/nature.yaml'
)

In [13]:
selector.health_index_generator.asymptomatic_ratio

0.2

# Adding the interaction

In [14]:
interaction = Interaction.from_file()

Beta are the intensities of the interaction taking place at the different groups

In [15]:
interaction.beta

{'box': 1,
 'pub': 0.1,
 'grocery': 0.1,
 'cinema': 0.1,
 'commute_unit': 1.0,
 'commute_city_unit': 1.0,
 'hospital': 1.0,
 'care_home': 1.0,
 'company': 0.1,
 'school': 0.1,
 'household': 0.2,
 'university': 0.01}

to modify these, simply do

In [16]:
for key in interaction.beta.keys():
    interaction.beta[key] *= 2

In [17]:
interaction.beta

{'box': 2,
 'pub': 0.2,
 'grocery': 0.2,
 'cinema': 0.2,
 'commute_unit': 2.0,
 'commute_city_unit': 2.0,
 'hospital': 2.0,
 'care_home': 2.0,
 'company': 0.2,
 'school': 0.2,
 'household': 0.4,
 'university': 0.02}

moreover this interaction module uses contact matrices, that are different for different groups. These contact matrices shouldnt be modified for now. However they are a combination of conversational contact matrices, and physical contact matrices (see the BBC pandemic paper, from where these matrices are extracted https://www.medrxiv.org/content/10.1101/2020.02.16.20023754v2)

There is a parameter, ``alpha`` ($\alpha$), that combines these two matrices in the following way,


$\beta M \left(1 + (\alpha -1) \right) P$

where $\beta$ is the intensity of the interaction, and $P$ the physical contact matrix. A larger $\alpha$ produces more physical contacts. It is an overall number, non dependent of the particular group.


In [18]:
interaction.alpha_physical

2.0

# Seed the disease

There are two options implemented in the seed at the moment, either you specify the number of cases and these are then homogeneously distributed by population to the different areas, or you use UK data on cases per region. For now use the first case.

In [19]:
infection_seed = InfectionSeed(
    world.super_areas, selector,
)

In [20]:
n_cases = 50
infection_seed.unleash_virus(n_cases) # play around with the initial number of cases

# Set policies

In [21]:
policies = Policies.from_file()

In [22]:
policies

# Run the simulation

The simulator is the main module in charge of running the simulation. It coordinates the ``ActivityManager`` which is responsible of allocating people to the right groups given the current timestep, it updates the health status of the population, and it runs the interaction over the different groups. All of these modules can be modified by policies at any given time.

Since the timer configuration is a bit cumbersome, it is read from the config file at ``configs/config_example.yaml``

In [23]:
simulator = Simulator.from_file(
    world=world,
    infection_selector=selector,
    interaction=interaction, 
    config_filename = CONFIG_PATH,
    leisure = leisure,
    policies = policies
)

In [ ]:
%%time
simulator.run()

2020-08-04 15:50:45,076 - june.simulator - INFO - Starting group_dynamics for 210 days at day 0
2020-08-04 15:50:45,079 - june.simulator - INFO - starting the loop ..., at 0 days, to run for 210 days
2020-08-04 15:50:46,492 - june.simulator - INFO - Date = 2020-03-01 04:00:00, number of deaths =  0, number of infected = 50
2020-08-04 15:50:47,084 - june.simulator - INFO - Date = 2020-03-01 08:00:00, number of deaths =  0, number of infected = 50
2020-08-04 15:50:47,409 - june.simulator - INFO - Date = 2020-03-01 12:00:00, number of deaths =  0, number of infected = 50
2020-08-04 15:50:47,733 - june.simulator - INFO - Date = 2020-03-02 00:00:00, number of deaths =  0, number of infected = 50
2020-08-04 15:50:48,137 - june.simulator - INFO - Date = 2020-03-02 01:00:00, number of deaths =  0, number of infected = 50
2020-08-04 15:50:48,735 - june.simulator - INFO - Date = 2020-03-02 09:00:00, number of deaths =  0, number of infected = 51
2020-08-04 15:50:49,328 - june.simulator - INFO - 

2020-08-04 15:51:36,747 - june.simulator - INFO - Date = 2020-03-14 12:00:00, number of deaths =  0, number of infected = 2448
2020-08-04 15:51:38,658 - june.simulator - INFO - Date = 2020-03-15 00:00:00, number of deaths =  0, number of infected = 2618
2020-08-04 15:51:40,679 - june.simulator - INFO - Date = 2020-03-15 04:00:00, number of deaths =  0, number of infected = 2750
2020-08-04 15:51:42,991 - june.simulator - INFO - Date = 2020-03-15 08:00:00, number of deaths =  0, number of infected = 2964
2020-08-04 15:51:44,719 - june.simulator - INFO - Date = 2020-03-15 12:00:00, number of deaths =  0, number of infected = 3070
2020-08-04 15:51:47,821 - june.simulator - INFO - Date = 2020-03-16 00:00:00, number of deaths =  0, number of infected = 3267
2020-08-04 15:51:49,472 - june.simulator - INFO - Date = 2020-03-16 01:00:00, number of deaths =  0, number of infected = 3332
2020-08-04 15:51:52,061 - june.simulator - INFO - Date = 2020-03-16 09:00:00, number of deaths =  0, number of 

2020-08-04 15:54:54,938 - june.simulator - INFO - Date = 2020-03-28 08:00:00, number of deaths =  10, number of infected = 11031
2020-08-04 15:54:59,090 - june.simulator - INFO - Date = 2020-03-28 12:00:00, number of deaths =  13, number of infected = 11159
2020-08-04 15:55:03,788 - june.simulator - INFO - Date = 2020-03-29 00:00:00, number of deaths =  15, number of infected = 11370
2020-08-04 15:55:07,494 - june.simulator - INFO - Date = 2020-03-29 04:00:00, number of deaths =  15, number of infected = 11484
2020-08-04 15:55:11,195 - june.simulator - INFO - Date = 2020-03-29 08:00:00, number of deaths =  15, number of infected = 11593
2020-08-04 15:55:15,327 - june.simulator - INFO - Date = 2020-03-29 12:00:00, number of deaths =  15, number of infected = 11716
2020-08-04 15:55:20,084 - june.simulator - INFO - Date = 2020-03-30 00:00:00, number of deaths =  15, number of infected = 11924
2020-08-04 15:55:23,018 - june.simulator - INFO - Date = 2020-03-30 01:00:00, number of deaths = 

2020-08-04 15:59:59,646 - june.simulator - INFO - Date = 2020-04-11 00:00:00, number of deaths =  42, number of infected = 14726
2020-08-04 16:00:02,909 - june.simulator - INFO - Date = 2020-04-11 04:00:00, number of deaths =  42, number of infected = 14685
2020-08-04 16:00:06,819 - june.simulator - INFO - Date = 2020-04-11 08:00:00, number of deaths =  42, number of infected = 14665
2020-08-04 16:00:11,739 - june.simulator - INFO - Date = 2020-04-11 12:00:00, number of deaths =  43, number of infected = 14636
2020-08-04 16:00:17,291 - june.simulator - INFO - Date = 2020-04-12 00:00:00, number of deaths =  44, number of infected = 14511
2020-08-04 16:00:21,934 - june.simulator - INFO - Date = 2020-04-12 04:00:00, number of deaths =  44, number of infected = 14478
2020-08-04 16:00:26,216 - june.simulator - INFO - Date = 2020-04-12 08:00:00, number of deaths =  44, number of infected = 14429
2020-08-04 16:00:30,132 - june.simulator - INFO - Date = 2020-04-12 12:00:00, number of deaths = 

2020-08-04 16:03:50,281 - june.simulator - INFO - Date = 2020-04-24 10:00:00, number of deaths =  56, number of infected = 9864
2020-08-04 16:03:52,517 - june.simulator - INFO - Date = 2020-04-24 13:00:00, number of deaths =  56, number of infected = 9799
2020-08-04 16:03:55,240 - june.simulator - INFO - Date = 2020-04-25 00:00:00, number of deaths =  57, number of infected = 9570
2020-08-04 16:03:57,817 - june.simulator - INFO - Date = 2020-04-25 04:00:00, number of deaths =  57, number of infected = 9505
2020-08-04 16:04:00,275 - june.simulator - INFO - Date = 2020-04-25 08:00:00, number of deaths =  58, number of infected = 9400
2020-08-04 16:04:02,584 - june.simulator - INFO - Date = 2020-04-25 12:00:00, number of deaths =  58, number of infected = 9311
2020-08-04 16:04:05,252 - june.simulator - INFO - Date = 2020-04-26 00:00:00, number of deaths =  58, number of infected = 9064
2020-08-04 16:04:08,370 - june.simulator - INFO - Date = 2020-04-26 04:00:00, number of deaths =  58, nu

2020-08-04 16:06:07,095 - june.simulator - INFO - Date = 2020-05-08 09:00:00, number of deaths =  69, number of infected = 4138
2020-08-04 16:06:08,382 - june.simulator - INFO - Date = 2020-05-08 10:00:00, number of deaths =  69, number of infected = 4136
2020-08-04 16:06:09,570 - june.simulator - INFO - Date = 2020-05-08 13:00:00, number of deaths =  69, number of infected = 4105
2020-08-04 16:06:10,897 - june.simulator - INFO - Date = 2020-05-09 00:00:00, number of deaths =  69, number of infected = 3962
2020-08-04 16:06:12,230 - june.simulator - INFO - Date = 2020-05-09 04:00:00, number of deaths =  69, number of infected = 3918
2020-08-04 16:06:14,124 - june.simulator - INFO - Date = 2020-05-09 08:00:00, number of deaths =  69, number of infected = 3886
2020-08-04 16:06:16,287 - june.simulator - INFO - Date = 2020-05-09 12:00:00, number of deaths =  69, number of infected = 3856
2020-08-04 16:06:17,863 - june.simulator - INFO - Date = 2020-05-10 00:00:00, number of deaths =  69, nu

2020-08-04 16:07:24,008 - june.simulator - INFO - Date = 2020-05-22 01:00:00, number of deaths =  79, number of infected = 1486
2020-08-04 16:07:24,861 - june.simulator - INFO - Date = 2020-05-22 09:00:00, number of deaths =  79, number of infected = 1459
2020-08-04 16:07:25,793 - june.simulator - INFO - Date = 2020-05-22 10:00:00, number of deaths =  79, number of infected = 1455
2020-08-04 16:07:26,688 - june.simulator - INFO - Date = 2020-05-22 13:00:00, number of deaths =  79, number of infected = 1445
2020-08-04 16:07:27,668 - june.simulator - INFO - Date = 2020-05-23 00:00:00, number of deaths =  79, number of infected = 1393
2020-08-04 16:07:28,726 - june.simulator - INFO - Date = 2020-05-23 04:00:00, number of deaths =  79, number of infected = 1378
2020-08-04 16:07:29,784 - june.simulator - INFO - Date = 2020-05-23 08:00:00, number of deaths =  79, number of infected = 1361
2020-08-04 16:07:30,850 - june.simulator - INFO - Date = 2020-05-23 12:00:00, number of deaths =  79, nu

2020-08-04 16:08:18,216 - june.simulator - INFO - Date = 2020-06-05 00:00:00, number of deaths =  80, number of infected = 549
2020-08-04 16:08:19,032 - june.simulator - INFO - Date = 2020-06-05 01:00:00, number of deaths =  80, number of infected = 547
2020-08-04 16:08:19,634 - june.simulator - INFO - Date = 2020-06-05 09:00:00, number of deaths =  80, number of infected = 534
2020-08-04 16:08:20,347 - june.simulator - INFO - Date = 2020-06-05 10:00:00, number of deaths =  80, number of infected = 531
2020-08-04 16:08:20,925 - june.simulator - INFO - Date = 2020-06-05 13:00:00, number of deaths =  80, number of infected = 524
2020-08-04 16:08:21,671 - june.simulator - INFO - Date = 2020-06-06 00:00:00, number of deaths =  80, number of infected = 497
2020-08-04 16:08:22,396 - june.simulator - INFO - Date = 2020-06-06 04:00:00, number of deaths =  80, number of infected = 483
2020-08-04 16:08:23,149 - june.simulator - INFO - Date = 2020-06-06 08:00:00, number of deaths =  80, number of

2020-08-04 16:09:05,331 - june.simulator - INFO - Date = 2020-06-18 13:00:00, number of deaths =  82, number of infected = 180
2020-08-04 16:09:05,894 - june.simulator - INFO - Date = 2020-06-19 00:00:00, number of deaths =  82, number of infected = 172
2020-08-04 16:09:06,661 - june.simulator - INFO - Date = 2020-06-19 01:00:00, number of deaths =  82, number of infected = 172
2020-08-04 16:09:07,209 - june.simulator - INFO - Date = 2020-06-19 09:00:00, number of deaths =  82, number of infected = 169
2020-08-04 16:09:07,896 - june.simulator - INFO - Date = 2020-06-19 10:00:00, number of deaths =  82, number of infected = 166
2020-08-04 16:09:08,417 - june.simulator - INFO - Date = 2020-06-19 13:00:00, number of deaths =  82, number of infected = 165
2020-08-04 16:09:09,141 - june.simulator - INFO - Date = 2020-06-20 00:00:00, number of deaths =  82, number of infected = 163
2020-08-04 16:09:09,872 - june.simulator - INFO - Date = 2020-06-20 04:00:00, number of deaths =  82, number of

2020-08-04 16:09:48,996 - june.simulator - INFO - Date = 2020-07-02 10:00:00, number of deaths =  82, number of infected = 77
2020-08-04 16:09:49,504 - june.simulator - INFO - Date = 2020-07-02 13:00:00, number of deaths =  83, number of infected = 76
2020-08-04 16:09:50,027 - june.simulator - INFO - Date = 2020-07-03 00:00:00, number of deaths =  83, number of infected = 76
2020-08-04 16:09:50,759 - june.simulator - INFO - Date = 2020-07-03 01:00:00, number of deaths =  83, number of infected = 76
2020-08-04 16:09:51,316 - june.simulator - INFO - Date = 2020-07-03 09:00:00, number of deaths =  83, number of infected = 75
2020-08-04 16:09:52,062 - june.simulator - INFO - Date = 2020-07-03 10:00:00, number of deaths =  83, number of infected = 75
2020-08-04 16:09:52,624 - june.simulator - INFO - Date = 2020-07-03 13:00:00, number of deaths =  83, number of infected = 76
2020-08-04 16:09:53,391 - june.simulator - INFO - Date = 2020-07-04 00:00:00, number of deaths =  83, number of infect

While the simulation runs (and afterwards) we can launch the visualization webpage by running
```python june/visualizer.py path/to/results``` 

# Getting the results

All results are stored in a json file specified in the ``save_path`` variable in the config file. We can also access it from ``world.logger`` directly.

In [ ]:
import pandas as pd

In [ ]:
read = ReadLogger()

## Hospital data and how it changed over time

In [ ]:
hospitals_df = read.load_hospital_capacity()

In [ ]:
hospitals_df.head(3)

In [ ]:
hospitals_characteristics_df = read.load_hospital_characteristics()

In [ ]:
hospitals_characteristics_df

## where did infections happen?

In [ ]:
loc_df = read.get_locations_infections()

In [ ]:
locations_per_day = read.locations_df.groupby(pd.Grouper(freq='D')).sum()

In [ ]:
all_infection_places = set(locations_per_day.location.sum())

In [ ]:
def n_infections(row, infection_place):
    return sum([row.counts[i] for i, x in enumerate(row.location) if x == infection_place])


In [ ]:
for infection_place in all_infection_places:
    locations_per_day[str(infection_place)] = locations_per_day.apply(
        lambda x: n_infections(x, infection_place),
        axis=1
    )

In [ ]:
locations_per_day = locations_per_day.drop(columns=['location',
                                                    'counts'])

In [ ]:
locations_per_day = locations_per_day.div(
    locations_per_day.sum(axis=1), axis=0
)


In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 100

In [ ]:
locations_per_day.plot.area( alpha=0.5)
plt.legend(bbox_to_anchor=(1,1))
plt.ylabel('Percent of infections')

In [ ]:
from matplotlib import cm
cmap = cm.get_cmap('Spectral') # Colour map (there are many others)

locations_per_day.plot(figsize=(30,8), logy=True, ylim=(1e-3, 1), cmap=cmap)

In [ ]:
import matplotlib.ticker as mtick

ax = loc_df['percentage_infections'].sort_values().plot.bar()
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
plt.ylabel('Percentage of infections at location')
plt.xlabel('location')
#plt.yscale('log')

## rate of infection

In [ ]:
r_df = read.get_r()

In [ ]:
from june.policy import SocialDistancing
r_df.plot()
colors = ["yellow", "orange"]
for policy in policies.policies:
    if isinstance(policy, SocialDistancing):
        plt.axvspan(policy.start_time, 
                policy.end_time,
                facecolor=colors.pop(), alpha=0.2)
plt.axhline(y=1, linestyle='dashed', color='gray')
plt.xlabel('Date')
plt.yscale('log')
plt.ylabel('R')

## World infection curves, and by super area

In [ ]:
world_df = read.world_summary()

In [ ]:
fig, ax = plt.subplots()
world_df['new_infections'].resample('D').sum().plot(ax=ax)
colors = ["yellow", "orange"]
for policy in policies.policies:
    if isinstance(policy, SocialDistancing):
        plt.axvspan(policy.start_time, 
                policy.end_time,
                facecolor=colors.pop(), alpha=0.2)
plt.axvline('2020-03-23',
           linestyle='dashed',
           color='indianred')
ax.set_ylabel('# new infections')
ax.set_xlabel('Date')

In [ ]:
fig, ax = plt.subplots()
(world_df['hospital_admissions']+world_df['intensive_care_admissions']).resample('D').sum().plot(ax=ax)
colors = ["yellow", "orange"]
for policy in policies.policies:
    if isinstance(policy, SocialDistancing):
        plt.axvspan(policy.start_time, 
                policy.end_time,
                facecolor=colors.pop(), alpha=0.2)
plt.axvline('2020-04-01',
           linestyle='dashed',
           color='indianred')
ax.set_ylabel('# hospital admissions')
ax.set_xlabel('Date')

In [ ]:
fig, ax = plt.subplots()
world_df['infected'].plot(ax=ax)
colors = ["yellow", "orange"]
for policy in policies.policies:
    if isinstance(policy, SocialDistancing):
        plt.axvspan(policy.start_time, 
                policy.end_time,
                facecolor=colors.pop(), alpha=0.2)
ax.set_ylabel('# Infected')
ax.set_xlabel('Date')

In [ ]:
world_df.plot()
colors = ["yellow", "orange"]
for policy in policies.policies:
    if isinstance(policy, SocialDistancing):
        plt.axvspan(policy.start_time, 
                policy.end_time,
                facecolor=colors.pop(), alpha=0.2)
plt.legend(bbox_to_anchor=(1,1))


In [ ]:
area_df = read.super_area_summary()

In [ ]:
area_df[area_df.super_area == "E02003282"].plot()
plt.legend(bbox_to_anchor=(1,1))

## World infection curves per age group

In [ ]:
ages_df = read.age_summary([0,10,20,30,40,
                  50,60,70,80,90,100])

In [ ]:
for name, group in ages_df.groupby('age_range'):
    group['infected'].plot(label=name)
colors = ["yellow", "orange"]
for policy in policies.policies:
    if isinstance(policy, SocialDistancing):
        plt.axvspan(policy.start_time, 
                policy.end_time,
                facecolor=colors.pop(), alpha=0.2)
plt.legend(bbox_to_anchor=(1,1))


In [ ]:
for name, group in ages_df.groupby('age_range'):
    group['dead'].cumsum().plot(label=name)
plt.legend()

In [ ]:
for name, group in ages_df.groupby('age_range'):
    group['susceptible'].plot(label=name)
plt.legend()

In [ ]:
for name, group in ages_df.groupby('age_range'):
    group['hospital_admissions'].cumsum().plot(label=name)
plt.legend()

In [ ]:
for name, group in ages_df.groupby('age_range'):
    group['intensive_care_admissions'].cumsum().plot(label=name)
plt.legend()

## Draw some of the symptoms trajectories

In [ ]:
random_trajectories = read.draw_symptom_trajectories(window_length=600,
                                        n_people=10)

In [ ]:
from june.infection import SymptomTag

In [ ]:
symptoms_values = [tag.value for tag in SymptomTag]
symptoms_names = [tag.name for tag in SymptomTag]


In [ ]:
for df_person in random_trajectories:
    df_person['symptoms'].plot()
plt.ylabel('Symptoms Trajectory')
_ = plt.yticks(symptoms_values, symptoms_names)
plt.xlabel('Date')

In [ ]:
for df_person in random_trajectories:
    df_person['n_secondary_infections'].plot()
plt.ylabel('Number of secondary infections')

plt.xlabel('Date')